In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

dacs_all=pd.read_csv(r"D:\MentalHealth\dacs_91_sleep_data.csv")

dacs_all['start_date']=pd.to_datetime(dacs_all['start_date'])
dacs_all['end_date']=pd.to_datetime(dacs_all['end_date'])

# add 'sleep efficiency' column
dacs_all['sleep_efficiency']=dacs_all['sleep_duration']/dacs_all['bed_duration']

# clean some useless columns
dacs_all = dacs_all.drop(['sleep_data_id','start_time','end_time'], axis=1)

# remove the unappeared users in Apr from dataframe, and re-run the previous lines to check  
dacs_all = dacs_all[(dacs_all.PID !='3-195') & (dacs_all.PID !='3-37') &\
                    (dacs_all.PID !='3-13') & (dacs_all.PID !='3-41') &\
                    (dacs_all.PID !='3-125') & (dacs_all.PID !='3-187') &\
                    (dacs_all.PID !='3-3') & (dacs_all.PID !='3-169') &\
                    (dacs_all.PID !='3-85') & (dacs_all.PID !='3-121') &\
                    (dacs_all.PID !='3-51') ]

In [2]:
###################################################
# make a boolean mask of window
###################################################

def get_masked_dataframe(start_date, end_date, df):
    mask = (df['start_date']>= start_date) & (df['start_date'] < end_date)
    new_df = df.loc[mask]
    new_df = new_df.set_index('start_date')
    return new_df

start_date = dt.strptime('2020-02-01 00:00:00','%Y-%m-%d %H:%M:%S')
end_date = dt.strptime('2020-05-02 00:00:00','%Y-%m-%d %H:%M:%S')
new_all = get_masked_dataframe(start_date, end_date, dacs_all)


list_each_user=[]
for name,group in new_all.groupby(['PID']):
    list_each_user.append(group)


In [3]:
# Feb as baseline
end_date_b = dt.strptime('2020-03-01 00:00:00','%Y-%m-%d %H:%M:%S')
feb_baseline_all = get_masked_dataframe(start_date, end_date_b, dacs_all)
#----------------------------------------
# Mar as a whole
end_date_mar = dt.strptime('2020-04-01 00:00:00','%Y-%m-%d %H:%M:%S')
mar_whole_month = get_masked_dataframe(end_date_b, end_date_mar, dacs_all)
#----------------------------------------
# Apr as a whole
end_date_apr = dt.strptime('2020-05-01 00:00:00','%Y-%m-%d %H:%M:%S')
apr_whole_month = get_masked_dataframe(end_date_mar, end_date_apr, dacs_all)

In [4]:
###################################################
# using pandas rolling window to get the mean and std
###################################################
# to use indow as offset, change the index into datetime

def one_week_sleep_parameters_mean(aaa,win,mp):
    '''
    aaa: each_user dataframe, with index as start_date
    mp: an int for min_periods
    win: window offset
    '''
    # any NAN in the results indicate the previous window contain 'Nan' 
    row1 = aaa['sleep_duration'].rolling(window = win,min_periods=mp).mean()
    row2 = aaa['bed_duration'].rolling(window = win,min_periods=mp).mean()
    row3 = aaa['sleep_score'].rolling(window = win,min_periods=mp).mean()
    row4 = aaa['awake_duration'].rolling(window = win,min_periods=mp).mean()
    row5 = aaa['rem_sleep_duration'].rolling(window = win,min_periods=mp).mean()
    row6 = aaa['light_sleep_duration'].rolling(window = win,min_periods=mp).mean()
    row7 = aaa['deep_sleep_duration'].rolling(window = win,min_periods=mp).mean()
    row8 = aaa['sleep_onset_duration'].rolling(window = win,min_periods=mp).mean()
    row9 = aaa['bed_exit_count'].rolling(window = win,min_periods=mp).mean()
    row10 = aaa['toss_turn_count'].rolling(window = win,min_periods=mp).mean()
    row11 = aaa['average_heart_rate'].rolling(window = win,min_periods=mp).mean()
    row12 = aaa['min_heart_rate'].rolling(window = win,min_periods=mp).mean()
    row13 = aaa['max_heart_rate'].rolling(window = win,min_periods=mp).mean()
    row14 = aaa['average_respiration_rate'].rolling(window = win,min_periods=mp).mean()
    row15 = aaa['min_respiration_rate'].rolling(window = win,min_periods=mp).mean()
    row16 = aaa['max_respiration_rate'].rolling(window = win,min_periods=mp).mean()
    row17 = aaa['average_physical_activity'].rolling(window = win,min_periods=mp).mean()
    row18 = aaa['fm_count'].rolling(window = win,min_periods=mp).mean()
    row19 = aaa['awakenings'].rolling(window = win,min_periods=mp).mean()
    row20 = aaa['hrv_score'].rolling(window = win,min_periods=mp).mean()
    row21 = aaa['hrv_lf'].rolling(window = win,min_periods=mp).mean()
    row22 = aaa['hrv_hf'].rolling(window = win,min_periods=mp).mean()
    row23 = aaa['hrv_rmssd_evening'].rolling(window = win,min_periods=mp).mean()
    row24 = aaa['hrv_rmssd_morning'].rolling(window = win,min_periods=mp).mean()
    row25 = aaa['sleep_efficiency'].rolling(window = win,min_periods=mp).mean()
    
    series_list = [row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, 
     row12, row13, row14, row15, row16, row17, row18, row19, row20, row21,
     row22, row23, row24, row25]
    
    dataframe_with_startdate_index = []
    for each_series in series_list:
        convert_to_dataframe = each_series.to_frame()
        dataframe_with_startdate_index.append(convert_to_dataframe)
    
    aa1 = pd.concat(dataframe_with_startdate_index,axis=1)
    return aa1

In [5]:
rolling_mean_list = []
for each_PID_df in list_each_user:
     each_PID_rolling_mean = one_week_sleep_parameters_mean(each_PID_df.sort_index(),'7D',6)
     rolling_mean_list.append(each_PID_rolling_mean)

In [6]:
def one_week_sleep_parameters_std(aaa,win,mp):
    '''
    aaa: each_user dataframe, with index as start_date
    mp: an int for min_periods
    win: window offset
    '''
    # any NAN in the results indicate the previous window contain 'Nan' 
    row1 = aaa['sleep_duration'].rolling(window = win,min_periods=mp).std()
    row2 = aaa['bed_duration'].rolling(window = win,min_periods=mp).std()
    row3 = aaa['sleep_score'].rolling(window = win,min_periods=mp).std()
    row4 = aaa['awake_duration'].rolling(window = win,min_periods=mp).std()
    row5 = aaa['rem_sleep_duration'].rolling(window = win,min_periods=mp).std()
    row6 = aaa['light_sleep_duration'].rolling(window = win,min_periods=mp).std()
    row7 = aaa['deep_sleep_duration'].rolling(window = win,min_periods=mp).std()
    row8 = aaa['sleep_onset_duration'].rolling(window = win,min_periods=mp).std()
    row9 = aaa['bed_exit_count'].rolling(window = win,min_periods=mp).std()
    row10 = aaa['toss_turn_count'].rolling(window = win,min_periods=mp).std()
    row11 = aaa['average_heart_rate'].rolling(window = win,min_periods=mp).std()
    row12 = aaa['min_heart_rate'].rolling(window = win,min_periods=mp).std()
    row13 = aaa['max_heart_rate'].rolling(window = win,min_periods=mp).std()
    row14 = aaa['average_respiration_rate'].rolling(window = win,min_periods=mp).std()
    row15 = aaa['min_respiration_rate'].rolling(window = win,min_periods=mp).std()
    row16 = aaa['max_respiration_rate'].rolling(window = win,min_periods=mp).std()
    row17 = aaa['average_physical_activity'].rolling(window = win,min_periods=mp).std()
    row18 = aaa['fm_count'].rolling(window = win,min_periods=mp).std()
    row19 = aaa['awakenings'].rolling(window = win,min_periods=mp).std()
    row20 = aaa['hrv_score'].rolling(window = win,min_periods=mp).std()
    row21 = aaa['hrv_lf'].rolling(window = win,min_periods=mp).std()
    row22 = aaa['hrv_hf'].rolling(window = win,min_periods=mp).std()
    row23 = aaa['hrv_rmssd_evening'].rolling(window = win,min_periods=mp).std()
    row24 = aaa['hrv_rmssd_morning'].rolling(window = win,min_periods=mp).std()
    row25 = aaa['sleep_efficiency'].rolling(window = win,min_periods=mp).std()
    
    series_list = [row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, 
     row12, row13, row14, row15, row16, row17, row18, row19, row20, row21,
     row22, row23, row24, row25]
    
    dataframe_with_startdate_index = []
    for each_series in series_list:
        convert_to_dataframe = each_series.to_frame()
        dataframe_with_startdate_index.append(convert_to_dataframe)
    
    aa1 = pd.concat(dataframe_with_startdate_index,axis=1)
    return aa1

In [7]:
rolling_std_list = []
for each_PID_df in list_each_user:
     each_PID_rolling_std = one_week_sleep_parameters_std(each_PID_df.sort_index(),'7D',6)
     rolling_std_list.append(each_PID_rolling_std)

In [17]:
# add the PID and start_date back to each dataframe
user_list=[]
for name,group in new_all.groupby(['PID']):
    user_list.append(name)

time_index_list=[]
for each_user in list_each_user:
    time_index = sorted(each_user.index.tolist())
    time_index_list.append(time_index)

for i in range(len(time_index_list)):
    each_mean = rolling_std_list[i] # dataframe
    each_user_time = time_index_list[i] # list
    each_mean['start_time']=each_user_time
    each_mean['PID']=user_list[i]       

for i in range(len(time_index_list)):
    each_mean = rolling_mean_list[i] # dataframe
    each_user_time = time_index_list[i] # list
    each_mean['start_time']=each_user_time
    each_mean['PID']=user_list[i] 

In [19]:
# The length of rolling_std_list and rolling_mean_list are 61, which reprsent 61 users that have sleep data up to Apr

rolling_mean_list[0].reset_index(drop=True).style.background_gradient(cmap='Reds')

In [20]:
rolling_std_list[0].reset_index(drop=True).style.background_gradient(cmap='Blues')

In [21]:
rolling_mean_list[1].reset_index(drop=True).style.background_gradient(cmap='Reds')

In [23]:
rolling_std_list[1].reset_index(drop=True).style.background_gradient(cmap='Blues')